In [ ]:
import os
import math
import json
import pickle
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.data import Dataset
from tensorflow import keras
from keras_vggface.vggface import VGGFace

from datetime import datetime
from matplotlib import pyplot as plt
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
from tensorflow.keras.applications import MobileNet, ResNet50, InceptionV3
from tensorflow.keras.applications.mobilenet import preprocess_input as mobilenet_preprocess
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_preprocess
from keras_vggface.utils import preprocess_input as vggface_preprocess
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing import image
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import Callback, LearningRateScheduler
from tensorflow_addons.optimizers import CyclicalLearningRate
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Lambda, \
Conv1D, Attention, GlobalAveragePooling1D, BatchNormalization, Concatenate, \
Layer, Reshape, Add, Multiply, Subtract
from inception_resnet_v1 import InceptionResNetV1
from keras_facenet import FaceNet

random.seed(123)
tf.random.set_seed(12)
np.random.seed(123)

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
TRAIN_PATH = './data/train'

# Training pairs generating

Available training pairs from csv files are splitted to train - validation sets. Those pairs are positive(there is blood relation). For each set(train/valid) we additionally generate negative pairs.

Positive pairs are generated according to the input csv file. For each person of positive pair we create one negative pair.
In total we'll have twice more negative than positive pairs.

In [ ]:
def make_image_pair(pair, input_shape, shuffle=True, slice_imgs=None):
    '''
    Create pair of embeddings.
    
    Arguments:
    p1, p2 -- paths to persons' images directories (familyID/personID)
    
    Returns:
    pairs -- array of image pairs, pairing is alligned to smaller number of images
    ''' 
    p1, p2 = [os.path.join(TRAIN_PATH, p) for p in pair]
    
    p1_imgs = os.listdir(p1)
    p2_imgs = os.listdir(p2)
    
    if shuffle:
        random.shuffle(p1_imgs)
        random.shuffle(p2_imgs)
    
    p1_imgs = p1_imgs[:slice_imgs]
    p2_imgs = p2_imgs[:slice_imgs]
    
    for i in range(len(p1_imgs)):
        for j in range(len(p2_imgs)):
            img1_path = os.path.join(p1, p1_imgs[i])
            img2_path = os.path.join(p2, p2_imgs[j])
            img1 = image.load_img(img1_path, target_size=(input_shape[0], input_shape[1]))
            img2 = image.load_img(img2_path, target_size=(input_shape[0], input_shape[1]))
            img1 = np.array(img1).astype('float32')
            img2 = np.array(img2).astype('float32')
            
            yield img1, img2

In [ ]:
def pairs_set(input_pairs, input_shape, shuffle=True, slice_imgs=None):
    for pair, label in input_pairs:
        try:
            img_pairs = make_image_pair(pair, input_shape, shuffle, slice_imgs)
            for img_pair in img_pairs:
                yield img_pair, label
        except (KeyError, FileNotFoundError):
            continue

def batched_pairs(input_pairs, batch_size, dataset_period, input_shape, preprocess, shuffle=True, slice_imgs=None):
    imgs1 = []
    imgs2 = []
    labels = []
    counter = 0
    for example in pairs_set(input_pairs, input_shape, shuffle, slice_imgs):
        # Get every nth sample
        counter += 1
        if counter % dataset_period:
            continue
        
        exmpls, label = example
        exmpl1, exmpl2 = exmpls
        imgs1.append(exmpl1)
        imgs2.append(exmpl2)
        labels.append(label)
        if len(labels) == batch_size:
            yield {'input_1':preprocess(np.array(imgs1)), 'input_2':preprocess(np.array(imgs2))}, np.array(labels).astype(float)
            imgs1, imgs2, labels = [], [], []

In [ ]:
with open('train_val_set.json', 'r') as f:
    train_val_set = json.load(f)

train_rlt_list, neg_train_rltshps, valid_rlt_list, neg_valid_rltshps = list(train_val_set.values())
train_rlt_list = train_rlt_list * 4

train_rlts = list(zip(train_rlt_list + neg_train_rltshps, [True]*len(train_rlt_list) + [False]*len(neg_train_rltshps)))
val_rlts = list(zip(valid_rlt_list + neg_valid_rltshps, [True]*len(valid_rlt_list) + [False]*len(neg_valid_rltshps)))

random.shuffle(train_rlts)
random.shuffle(val_rlts)

# Siamese network

Initial experimenting is done with conv1D deep neural network, as additional option for experimenting there is simple attention module.

In [ ]:
def prewhiten(x):
    if x.ndim == 4:
        axis = (1, 2, 3)
        size = x[0].size
    elif x.ndim == 3:
        axis = (0, 1, 2)
        size = x.size
    else:
        raise ValueError('Dimension should be 3 or 4')

    mean = np.mean(x, axis=axis, keepdims=True)
    std = np.std(x, axis=axis, keepdims=True)
    std_adj = np.maximum(std, 1.0/np.sqrt(size))
    y = (x - mean) / std_adj
    return y

def cross_correlation(vectors):
    '''
    The goal is to convolute outputs from both networks,
    each one from the batch of the first network output
    over appropriate one from another network output
    '''
    
    # x and y have shape: [batch_size, vector_size]
    x, y = vectors
    # We need to add 'channels' dimension -> [batch_size, vector_size, 1]
    x = tf.expand_dims(x, -1)
    y = tf.expand_dims(y, -1)
    
    # Firstly we do convolution as the second network
    # output vectors from the batch are all filters.
    # Technically, we've done convolution of all y over each x.
    # From documentation(https://www.tensorflow.org/api_docs/python/tf/nn/conv1d) desired shapes should be:
    # x shape: batch_shape(batch_size) + [in_width(vector_size), in_channels(1)]
    # y shape: [filter_width(vector_size), in_channels(1), out_channels(batch_size)]
    # Output shape is [batch_size, vector_length, batch_size].
    conv1d = tf.nn.conv1d(x, tf.transpose(y, perm=[1, 2, 0]), stride=1, padding='SAME')
    conv1d_shape = conv1d.shape
    
    # We need now to apply mask to the output in order to get desired
    # convolution results in the shape [batch_size, vector_length]
    bool_diag = tf.linalg.diag(tf.constant([True] * config.batch_size))
    mask = tf.repeat(tf.expand_dims(bool_diag, axis=1), conv1d_shape[1], axis=1)
    out = tf.squeeze(tf.ragged.boolean_mask(conv1d, mask), axis=-1)
    return out.to_tensor()

def mobilenet(input_shape, l2_value, dropout, model_name='mobilenet', trainable_after=None):
    mobile = MobileNet(
        input_shape=input_shape,
        dropout=dropout,
        include_top=False,
        pooling='avg',
        alpha=1.,
        weights=None
    )
    
    trainable = False
    for layer in mobile.layers:
        trainable = trainable or layer.name.startswith(trainable_after)
        layer.trainable = trainable
        if hasattr(layer, 'kernel_regularizer'):
            setattr(layer, 'kernel_regularizer', keras.regularizers.l2(l2_value))
        
    x = Dense(1024, kernel_regularizer=l2(l2_value), activation='relu')(mobile.output)
    x = Lambda(lambda x: K.l2_normalize(x,axis=1))(x)
    return Model(mobile.input, x)

def inception(input_shape, l2_value, dropout, model_name='inception', trainable_after=None):
    inception = InceptionV3(
        input_shape=input_shape,
        include_top=False,
        pooling='avg',
        weights='imagenet'
    )
    
    trainable = False
    for layer in inception.layers:
        trainable = trainable or layer.name.startswith(trainable_after)
        layer.trainable = trainable
        if hasattr(layer, 'kernel_regularizer'):
            setattr(layer, 'kernel_regularizer', keras.regularizers.l2(l2_value))
        
    x = Dense(128, kernel_regularizer=l2(l2_value), activation='relu')(inception.output)
    x = Lambda(lambda x: K.l2_normalize(x,axis=1))(x)
    return Model(inception.input, x)

def resnet50(input_shape, l2_value, dropout, model_name='resnet50', trainable_after=None):
    resnet = ResNet50(
        input_shape=input_shape,
        include_top=False,
        pooling='avg',
        weights='imagenet'
    )
    
    trainable = False
    for layer in resnet.layers:
        trainable = trainable or layer.name.startswith(trainable_after)
        layer.trainable = trainable
        if hasattr(layer, 'kernel_regularizer'):
            setattr(layer, 'kernel_regularizer', keras.regularizers.l2(l2_value))
        
    x = Dense(32, kernel_regularizer=l2(l2_value), activation='relu')(resnet.output)
    x = Lambda(lambda x: K.l2_normalize(x,axis=1))(x)
    return Model(resnet.input, x)

def vggface_resnet50(input_shape, l2_value, dropout, model_name='vgg', trainable_after=None):
    vggface_res = VGGFace(model='resnet50', include_top=False, input_shape=input_shape)
    
    trainable = False
    for layer in vggface_res.layers:
        trainable = trainable or layer.name.startswith(trainable_after)
        layer.trainable = trainable
        if hasattr(layer, 'kernel_regularizer'):
            setattr(layer, 'kernel_regularizer', keras.regularizers.l2(l2_value))
        if isinstance(layer, Dropout):
            layer.rate = dropout
    
    last_layer = vggface_res.get_layer('avg_pool').output
    
    # For independent instatiation
    x = Reshape((-1, 1))(last_layer)
    
    x = Conv1D(1, 128)(x)
    x = Conv1D(1, 128)(x)
    x = Flatten()(x)
    x = Dropout(dropout)(x)
    x = Lambda(lambda x: K.l2_normalize(x,axis=1))(x)

    # For instatiation as part of hybrid model
#     x = Flatten()(last_layer)
#     x = Dense(256, kernel_regularizer=l2(l2_value), activation='relu')(x)
#     x = Dropout(dropout)(x)
#     x = Lambda(lambda x: K.l2_normalize(x,axis=1))(x)
    
    return Model(vggface_res.input, x, name=model_name)

def facenet_inception_resnetv1(input_shape, l2_value, dropout, model_name='facenet', trainable_after=None):
    inception_resnet = InceptionResNetV1(input_shape=input_shape, weights_path='keras-facenet/weights/facenet_keras_weights.h5')
    
    trainable = False
    for layer in inception_resnet.layers:
        trainable = trainable or layer.name.startswith(trainable_after)
        layer.trainable = trainable
        if hasattr(layer, 'kernel_regularizer'):
            setattr(layer, 'kernel_regularizer', keras.regularizers.l2(l2_value))
        if isinstance(layer, Dropout):
            layer.rate = dropout
    
    last_layer = inception_resnet.get_layer('Dropout').output
    
    # For independent instatiation
    x = Reshape((-1, 1))(last_layer)
    
    x = Conv1D(1, 128)(x)
    x = Conv1D(1, 128)(x)
    x = Flatten()(x)
    x = Dropout(dropout)(x)
    x = Lambda(lambda x: K.l2_normalize(x,axis=1))(x)
    
    # For instatiation as part of hybrid model
#     x = Flatten()(last_layer)
#     x = Dense(256, kernel_regularizer=l2(l2_value), activation='relu')(x)
#     x = Dropout(dropout)(x)
# #     x = Lambda(lambda x: K.l2_normalize(x,axis=1))(x)
    
    return Model(inception_resnet.input, x, name=model_name)

def vgg_facenet(input_shape, l2_value, dropout, model_name='hybrid', trainable_after=None):
    vgg_model = vggface_resnet50(input_shape, l2_value, dropout, model_name='vgg', trainable_after=trainable_after)
    vgg_model = Model(vgg_model.input, vgg_model.layers[-2].output, name='vgg_reduced')
    facenet_model = facenet_inception_resnetv1(input_shape, l2_value, dropout, model_name='facenet', trainable_after=trainable_after)
    facenet_model = Model(facenet_model.input, facenet_model.layers[-2].output, name='facenet_reduced')
    
    model_input = Input(input_shape)
    vgg_out = vgg_model(model_input)
    facenet_out = facenet_model(model_input)
    
#     multiply = Multiply()([vgg_out, facenet_out])
#     multiply = Reshape((-1, 1))(multiply)
#     multiply = Conv1D(1, 13, padding='same')(multiply)
    
    add = Add()([vgg_out, facenet_out])
#     add = Reshape((-1, 1))(add)
#     add = Conv1D(1, 13, padding='same')(add)
    
#     sub = Subtract()([vgg_out, facenet_out])
#     sub_abs = Lambda(lambda x: K.abs(x))(sub)
#     sub_abs = Reshape((-1, 1))(sub_abs)
#     sub_abs = Conv1D(1, 13, padding='same')(sub_abs)
    
    output = Concatenate(axis=1)([vgg_out, facenet_out, add])#([multiply, add, sub_abs])
#     output = Reshape((-1, 1))(output)
    
#     output = Conv1D(1, 13, padding='same')(output)
#     output = Flatten()(output)
    
#     output = Dense(256, kernel_regularizer=l2(l2_value), activation='relu')(output)
#     output = Dropout(dropout)(output)
    output = Lambda(lambda x: K.l2_normalize(x,axis=1))(output)
    
    return Model(model_input, output, name=model_name)

# General configuration

In [ ]:
class Configuration:
    def __init__(self):
        # A number of images per person for
        # image-pair generation
        self.train_slice_persons_images = 4
        self.val_slice_persons_imges = 2
        
        # Factor for scaling variance loss in the total loss sum
        self.var_loss = 0.
        
        # Margin offset for positive and negative margins
        self.alpha = 0.2
        
        self.input_shape = (224, 224, 3)
        
        # Trainig parameters
        self.learning_rate = 5e-5
        self.l2_value = 1e-7
        self.dropout = 0.1
        self.epochs = 1000
        self.batch_size = 16
        self.model_type = 'mobilenet'
        self.preprocess = 'mobilenet_preprocess'
        
        # Sampling period of generated pairs
        self.train_dataset_period = 1
        self.eval_dataset_period = 1
        
        # Available distance types:
        # 'euclidean_distance', 'cosine_distance', 'cos_euc_distance'
        self.distance_type = 'cosine_distance'
        self.optimizer = 'Adam'
        
        # A layer name (or at least starting part of the name)
        # after which all layers are trainable
        self.trainable_after_layer = 'input'
        
        # Path to the pretrained model.
        # If there is no any, than None
        self.pretrained_base = 'pretrained/checkpoints/mobilenet_contrastive/model.hdf5'
        
config = Configuration()

# Learning rate scheduler
def scheduler(epoch, lr):
    if epoch == 10:
        return 0.5 * lr
    elif epoch == 180:
        return 0.8 * lr
    elif epoch == 250:
        return 0.5 * lr
    elif epoch == 300:
        return 0.7 * lr
    elif epoch == 400:
        return 0.3 *lr
    elif epoch == 700:
        return 0.6 *lr
    return lr
    
lr_callback = LearningRateScheduler(scheduler)

# Loss and metrics functions

In [ ]:
def get_margin(model):
    MARGIN = None
    for weight in model.layers[-1].weights:
        if weight.name == 'margin:0':
            MARGIN = weight
            break
    
    return MARGIN

def euclidean_distance(vectors):
    x, y = vectors
    sum_square = K.sum(K.square(x - y), axis=1)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

def cosine_distance(vectors):
    x, y = vectors
    x_norm = tf.norm(x, axis=1)
    y_norm = tf.norm(y, axis=1)
    x_y_dot = tf.einsum('ij,ij->i', x, y)
    cos_sim = x_y_dot / (x_norm * y_norm + K.epsilon())
    return 1. - cos_sim

def cos_euc_distance(vectors):
    euc = euclidean_distance(vectors)
    cos_dist = cosine_distance(vectors)
    return cos_dist * euc

def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

class ContrastiveLoss(keras.losses.Loss):
    def __init__(self, model, var_loss_factor):
        self.model = model
        self.var_loss_factor = var_loss_factor
        super(ContrastiveLoss, self).__init__()

    def call(self, y_true, y_pred):
        weight_pos = 1.
        weight_neg = 1.
        
        y_true = tf.squeeze(y_true, [1]) if len(y_true.shape) == 2 else y_true
        y_pred = tf.squeeze(y_pred, [1]) if len(y_pred.shape) == 2 else y_pred
        
        MARGIN = get_margin(self.model)
        margin_pos = MARGIN - config.alpha
        margin_neg = MARGIN + config.alpha

        # Filter predictions
        pred_pos = tf.boolean_mask(y_pred, y_true)
        pred_neg = tf.boolean_mask(y_pred, 1 - y_true)
        pred_pos_m = tf.reduce_mean(pred_pos, axis=0)
        pred_neg_m = tf.reduce_mean(pred_neg, axis=0)

        # Contrastive loss
        square_pos = tf.square(tf.maximum(pred_pos - margin_pos, 0))
        square_neg = tf.square(tf.maximum(margin_neg - pred_neg, 0))
        squared_concat = tf.concat([weight_pos * square_pos, weight_neg * square_neg], axis=0)

        # Variance loss
        pred_pos = tf.boolean_mask(pred_pos, pred_pos > margin_pos)
        pred_neg = tf.boolean_mask(pred_neg, pred_neg < margin_neg)
        pred_pos_m = tf.reduce_mean(pred_pos, axis=0)
        pred_neg_m = tf.reduce_mean(pred_neg, axis=0)
        var_pos = 0. if tf.equal(tf.size(pred_pos), 0) else tf.reduce_mean(tf.square(pred_pos_m - pred_pos))
        var_neg = 0. if tf.equal(tf.size(pred_neg), 0) else tf.reduce_mean(tf.square(pred_neg_m - pred_neg))
        variance_loss = 0.5 * (var_pos + var_neg)


        return tf.reduce_mean(squared_concat) + self.var_loss_factor * variance_loss

# Run training

In [ ]:
data_augmentation = keras.Sequential([
    keras.layers.experimental.preprocessing.RandomFlip("horizontal", seed=123),
    keras.layers.experimental.preprocessing.RandomTranslation(height_factor=0.2, width_factor=0.2, seed=123),
    keras.layers.experimental.preprocessing.RandomContrast(factor=0.2, seed=123),
    keras.layers.experimental.preprocessing.RandomZoom(height_factor=0.2, seed=123),
    keras.layers.experimental.preprocessing.RandomRotation(factor=0.1, seed=123)
])

In [ ]:
base_network = eval(config.model_type)(
    config.input_shape,
    config.l2_value,
    config.dropout,
    'base_network',
    trainable_after=config.trainable_after_layer
)

if config.pretrained_base:
    base_network.load_weights(config.pretrained_base)

print(f'Base network parameters count: {base_network.count_params()}')

In [ ]:
# Creation of Siamese network
input1 = Input(shape=config.input_shape, name='input_1')
input2 = Input(shape=config.input_shape, name='input_2')

input_1_aug = data_augmentation(input1)
input_2_aug = data_augmentation(input2)

processed1 = base_network(input_1_aug)
processed2 = base_network(input_2_aug)

In [ ]:
dist_function = eval(config.distance_type)
distance = Lambda(dist_function,
                  output_shape=eucl_dist_output_shape)([processed1, processed2])
model = Model([input1, input2], distance)

model.layers[len(model.layers) - 1].add_weight(
    name = 'margin',
    shape = (),
    initializer = keras.initializers.constant(0.5),
    trainable = True
)

## Run tensorboard plugin in order to track changes of training

In [ ]:
# Load the TensorBoard notebook extension
%reload_ext tensorboard

In [ ]:
%tensorboard --logdir=./logs --port=7008

## Training Callbacks

In [ ]:
# Get training set length
train_len = 0
for pair, label in train_rlts:
    try:
        p1, p2 = [os.path.join(TRAIN_PATH, p) for p in pair]
        p1_imgs = os.listdir(p1)
        p2_imgs = os.listdir(p2)
        train_len += len(p1_imgs[:config.train_slice_persons_images]) * \
                     len(p2_imgs[:config.train_slice_persons_images])
    except FileNotFoundError:
        continue

train_len = train_len // config.train_dataset_period

val_len = 0
for pair, label in val_rlts:
    try:
        p1, p2 = [os.path.join(TRAIN_PATH, p) for p in pair]
        p1_imgs = os.listdir(p1)
        p2_imgs = os.listdir(p2)
        val_len += len(p1_imgs[:config.val_slice_persons_imges]) * \
                     len(p2_imgs[:config.val_slice_persons_imges])
    except FileNotFoundError:
        continue

val_len = val_len // config.eval_dataset_period

print(f'Train set length: {train_len}')
print(f'Valid set length: {val_len}')

In [ ]:
def val_distance_stats(predictions, labels):
    val_pos = predictions[labels.astype(np.bool)]
    val_neg = predictions[(1 - labels).astype(np.bool)]
    val_pos_m, val_pos_s = np.mean(val_pos), np.std(val_pos)
    val_neg_m, val_neg_s = np.mean(val_neg), np.std(val_neg)
    
    return val_pos_m, val_pos_s, val_neg_m, val_neg_s
   
def dist_to_prob(predictions, lower_lim, upper_lim):
    y_prob = 1 - (np.clip(predictions, lower_lim, upper_lim) - lower_lim) / (upper_lim - lower_lim)
    return y_prob

def get_limits(MARGIN):
    # Get upper and lower boundary for the predicted distances
    margin_pos = MARGIN - config.alpha
    margin_neg = MARGIN + config.alpha
    lower_lim = max(MARGIN - 2. * (MARGIN - margin_pos), 0.)
    upper_lim = min(MARGIN + 2. * (margin_neg - MARGIN), 2.)
    
    return lower_lim, upper_lim
    
class MetricCallback(keras.callbacks.Callback):
    def __init__(self, logdir):
        super(MetricCallback, self).__init__()
        if not os.path.exists(logdir):
            os.makedirs(logdir)
        self.train_writer = tf.summary.create_file_writer(logdir + '/train')
        self.valid_writer = tf.summary.create_file_writer(logdir + '/valid')
        self.class_encoded = {
            0: 'not_related',
            1: 'related'
        }
        
    def tb_writer(self, items_to_write, wtype, epoch):
        writer = self.train_writer if wtype == 'train' else self.valid_writer
        
        with writer.as_default():
            for name, value in items_to_write.items():
                tf.summary.scalar(name, value, epoch)
            writer.flush()
        
    def on_epoch_end(self, epoch, logs={}):
        # Get current value of MARGIN
        MARGIN = get_margin(self.model).numpy()
        print(f'Current margin value: {MARGIN}')
        
        lower_lim, upper_lim = get_limits(MARGIN)
        
        val_true = []
        val_pred = []
        batches = batched_pairs(
            val_rlts,
            config.batch_size,
            config.eval_dataset_period,
            config.input_shape,
            eval(config.preprocess),
            shuffle=False,
            slice_imgs=config.val_slice_persons_imges
        )
        
        for batch in batches:
            pred = self.model.predict(batch[0])
            val_pred.append(pred)
            val_true.extend(list(batch[1]))
        
        val_true = np.array(val_true)
        val_pred = np.concatenate(val_pred, axis=0).squeeze()
        val_loss = ContrastiveLoss(self.model, config.var_loss)(K.constant(val_true), K.constant(val_pred))
        
        val_true = val_true.astype(int)
        val_pos_m, val_pos_s, val_neg_m, val_neg_s = val_distance_stats(val_pred, val_true)
        threshold = MARGIN
        
        # Precision and recall
        val_cls = (val_pred < threshold).astype(int)
        val_precision, val_recall, _, _ = precision_recall_fscore_support(val_true, val_cls, labels=[0, 1])
        val_accuracy = accuracy_score(val_true, val_cls)
        
        # Area under ROC
        val_probs = dist_to_prob(val_pred, lower_lim, upper_lim)
        val_roc_auc = roc_auc_score(val_true, val_probs)
        train_loss = logs['loss']
        tb_logs = {}
        tb_logs['train/loss'] = train_loss
        tb_logs['train/margin'] = MARGIN
        self.tb_writer(tb_logs, wtype='train', epoch=epoch)
        
        tb_logs = {}
        tb_logs['valid/loss'] = val_loss
        logs['val_loss'] = val_loss
        for k, v in self.class_encoded.items():
            tb_logs['valid/precision/' + v] = val_precision[k]
            tb_logs['valid/recall/' + v] = val_recall[k]
            tb_logs['valid/dist_mean/' + v] = val_pos_m if k else val_neg_m
            tb_logs['valid/dist_std/' + v] = val_pos_s if k else val_neg_s
        
        tb_logs['valid/accuracy'] = val_accuracy
        logs['val_roc_auc'] = val_roc_auc
        tb_logs['valid/roc_auc'] = val_roc_auc

        self.tb_writer(tb_logs, wtype='valid', epoch=epoch)

In [ ]:
optimizer = eval(config.optimizer)(learning_rate=config.learning_rate)
model.compile(loss=[ContrastiveLoss(model, config.var_loss)], optimizer=optimizer)
model.summary()

In [ ]:
model_name = 'mobilenet_001'

#Save training configuration
with open(f'configs/{model_name}.json', 'w') as f:
    json.dump(vars(config), f)

# Save architecture
keras.utils.plot_model(base_network, f'architectures/{model_name}.png', expand_nested=True, show_shapes=True)

logdir = os.path.join('logs', model_name)
ckpt_dir = os.path.join('checkpoints', model_name)
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

tensorboard_callback = keras.callbacks.TensorBoard(logdir, histogram_freq=1)
ckpt_callback = keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(ckpt_dir, 'model.hdf5'),
    monitor='val_roc_auc',
    save_best_only=True,
    mode='max',
    save_weights_only=False,
    verbose=1
)
metric_callback = MetricCallback(logdir)

In [ ]:
def repeat_generator(rlts, batch_size, dataset_period, input_shape, preprocess):
    while True:
        for e in batched_pairs(rlts, batch_size, dataset_period, input_shape, preprocess, config.train_slice_persons_images):
            yield e
            
model.fit(
    repeat_generator(train_rlts, config.batch_size, config.train_dataset_period, config.input_shape, eval(config.preprocess)),
    epochs=config.epochs,
    steps_per_epoch=train_len//config.batch_size,
    callbacks=[metric_callback, ckpt_callback]
)

# Submission

In [ ]:
# Load submission pairs
submission_path = 'data/sample_submission.csv'
submission_df = pd.read_csv(submission_path)

In [ ]:
# Load models
ckpt_path = 'checkpoints/vggface_002/model.hdf5'
model.load_weights(ckpt_path)
# model = keras.models.load_model(ckpt_path, custom_objects={'ContrastiveLoss': ContrastiveLoss})

In [ ]:
# Iterate over submission pairs
submission_df = submission_df.astype({'is_related': 'float'})
is_related = submission_df['is_related']
predictions = []
for idx, row in submission_df.iterrows():
    # Load images
    img_pair = row['img_pair']
    img1_name, img2_name = img_pair.split('-')
    img1_path = os.path.join('data/test', img1_name)
    img2_path = os.path.join('data/test', img2_name)
    img1 = image.load_img(img1_path, target_size=(config.input_shape[0], config.input_shape[1]))
    img2 = image.load_img(img2_path, target_size=(config.input_shape[0], config.input_shape[1]))
    img1 = eval(config.preprocess)(np.array(img1).astype('float32'))
    img2 = eval(config.preprocess)(np.array(img2).astype('float32'))
    img1 = np.expand_dims(img1, 0)
    img2 = np.expand_dims(img2, 0)
    
    # Do an inference, and calculate probability according to distance
    y_pred = model.predict({'input_1':img1, 'input_2':img2})
    y_pred = y_pred.squeeze()
    
    lower_bound, upper_bound = get_limits(get_margin(model))
    y_prob = dist_to_prob(y_pred, lower_bound, upper_bound)
    
    predictions.append(y_prob)
    is_related[idx] = y_prob
    
    # Print step
    if idx % 100 == 0:
        print(f'Processed rows: {idx}')
        
submission_df.to_csv(f'facenet_015.csv', index=False)

In [ ]:
plt.subplots(figsize=(30, 8))
plt.hist(predictions, 1000)
plt.locator_params(axis='x', nbins=100)
plt.xticks(rotation = 45)
plt.show()

In [ ]:
submission_df = pd.read_csv(submission_path)
is_related = submission_df['is_related']
# print(is_related.sum())
for i, pred in enumerate(predictions):
    if pred < 0.149:
        is_related[i] = 1
submission_df.to_csv(f'submission.csv', index=False)